## Overview
This notebook focuses on finetuning the terramind model to identify floods in a sentinel scene. The main take aways from this notebook will be as follows:
1. Learn how to use Terratorch to finetune terramind for floods in sentinel scene.
2. Understand the effects of spefic parameters in training and hardware utilization.

**Note:** The entirety of this notebook is tuned to work well in a sagemaker environment. Sagemaker Training Job will be used to train the models. If you are interested in running this in a colab environment, please leverage [ESA NASA Foundation Model Workshop](https://github.com/NASA-IMPACT/ESA-NASA-workshop-2025/tree/main/Track%201%20(EO)/TerraMind) materials.

## Setup
1. Go to "Kernel"
2. Select "terramind"

In [ ]:
import gdown
import lightning.pytorch as pl
import matplotlib.pyplot as plt
import numpy as np
import os
import rasterio
import sagemaker
import terratorch
import torch
import yaml

from datetime import time
from glob import glob
from huggingface_hub import hf_hub_download, snapshot_download
from pathlib import Path

from sagemaker import get_execution_role
from sagemaker.estimator import Estimator


3. Download the dataset from Google Drive

In [ ]:
!mkdir ../data/
if not os.path.isfile("../data/sen1floods11_v1.1.tar.gz"):
    gdown.download("https://drive.google.com/uc?id=1lRw3X7oFNq_WyzBO6uyUJijyTuYm23VS", '../data/sen1floods11_v1.1.tar.gz')

!tar -xzf ../data/sen1floods11_v1.1.tar.gz -C ../data/

In [ ]:
# Prepare sagemaker session with files uploaded to s3 bucket
dataset_path = Path("../data/sen1floods11_v1.1")
BUCKET_NAME = <BUCKET_NAME>

# Prepare sagemaker session with files uploaded to s3 bucket
sagemaker_session = sagemaker.Session()
all_files = sagemaker_session.upload_data(path=f'{dataset_path}', bucket=BUCKET_NAME, key_prefix='data')

In [7]:
# check config file.
import yaml
with open('../configs/terramind_v1_base_sen1floods11.yaml') as config_file:
    config = yaml.safe_load(config_file)

config

{'seed_everything': 42,
 'trainer': {'accelerator': 'auto',
  'strategy': 'auto',
  'devices': 'auto',
  'num_nodes': 1,
  'precision': '16-mixed',
  'logger': True,
  'callbacks': [{'class_path': 'RichProgressBar'},
   {'class_path': 'LearningRateMonitor',
    'init_args': {'logging_interval': 'epoch'}},
   {'class_path': 'ModelCheckpoint',
    'init_args': {'dirpath': '../output/burnscars/checkpoints',
     'mode': 'max',
     'monitor': 'val/Multiclass_Jaccard_Index',
     'filename': 'best-mIoU',
     'save_weights_only': True}}],
  'max_epochs': 100,
  'log_every_n_steps': 5,
  'default_root_dir': '../output/terramind_base_sen1floods11/'},
 'data': {'class_path': 'terratorch.datamodules.GenericMultiModalDataModule',
  'init_args': {'task': 'segmentation',
   'batch_size': 4,
   'num_workers': 4,
   'modalities': ['S2L1C', 'S1GRD'],
   'rgb_modality': 'S2L1C',
   'rgb_indices': [3, 2, 1],
   'train_data_root': {'S2L1C': '../data/sen1floods11_v1.1/data/S2L1CHand',
    'S1GRD': '../d

# TerraTorch model registry

TerraTorch includes its own backbone registry with many EO FMs. It also includes meta registries for all model components that include other sources like timm image models or SMP decoders.

In [ ]:
from terratorch.registry import BACKBONE_REGISTRY, TERRATORCH_BACKBONE_REGISTRY, TERRATORCH_DECODER_REGISTRY

**Note:** TiM models are using the Thinking-in-Modalities approach, see our [paper](https://arxiv.org/abs/2504.11171) for details.

In [ ]:
# Print all TerraMind v1 backbones.  
[backbone   
 for backbone in TERRATORCH_BACKBONE_REGISTRY
 if 'terramind_v1' in backbone]

In [ ]:
# Available decoders. We use the UNetDecoder in this example.
list(TERRATORCH_DECODER_REGISTRY)

In [ ]:
root_dir = '/opt/ml/data/'

# Parameters to modify
identifier = <identifier>

# Usecase definition
usecase = "terramind_base_sen1floods11"

#local data path
data_path = '../data/sen1floods11_v1.1'

# Output base directory
output_folder = f"{root_dir}/output"

# Default root dir for all data and splits
default_root_dir = f"{output_folder}/terramind_base_sen1floods11/"

# Path to save checkpoints in
checkpoint_path = f"{output_folder}/burnscars/checkpoints"

# Define standardization values. We use the pre-training values here and providing the additional modalities is not a problem, 
# which makes it simple to experiment with different modality combinations. 
# Alternatively, use the dataset statistics that you can generate using 
# `terratorch compute_statistics -c config.yaml` (requires concat_bands: true for this multimodal datamodule).
data_mean = {
    "S2L1C": [2357.089, 2137.385, 2018.788, 2082.986, 2295.651, 2854.537, 3122.849, 3040.560, 3306.481, 1473.847, 506.070, 2472.825, 1838.929],
    "S2L2A": [1390.458, 1503.317, 1718.197, 1853.910, 2199.100, 2779.975, 2987.011, 3083.234, 3132.220, 3162.988, 2424.884, 1857.648],
    "S1GRD": [-12.599, -20.293],
    "S1RTC": [-10.93, -17.329],
    "RGB": [87.271, 80.931, 66.667],
    "DEM": [670.665]
}

data_stds = {
    "S2L1C": [1624.683, 1675.806, 1557.708, 1833.702, 1823.738, 1733.977, 1732.131, 1679.732, 1727.26, 1024.687, 442.165, 1331.411, 1160.419],
    "S2L2A": [2106.761, 2141.107, 2038.973, 2134.138, 2085.321, 1889.926, 1820.257, 1871.918, 1753.829, 1797.379, 1434.261, 1334.311],
    "S1GRD": [5.195, 5.890],
    "S1RTC": [4.391, 4.459],
    "RGB": [58.767, 47.663, 42.631],
    "DEM": [951.272],
}

# Batches of examples to use at a time
batch_size = 4

# Number of training job workers
num_workers = 2

# Number of classes for segmentation
num_classes = 2


modalities = ["S2L1C", "S1GRD"]
rgb_modality = "S2L1C"  # Used for plotting. Defaults to the first modality if not provided.
rgb_indices = [3,2,1]  # RGB channel positions in the rgb_modality.

# Define data paths as dicts using the modality names as keys.
train_data_root = {
    "S2L1C": f"{default_root_dir}/S2L1CHand",
    "S1GRD": f"{default_root_dir}/S1GRDHand",
}
train_label_data_root = f"{default_root_dir}/LabelHand"
val_data_root = {
    "S2L1C": f"{default_root_dir}/S2L1CHand",
    "S1GRD": f"{default_root_dir}/S1GRDHand",
}
val_label_data_root = f"{default_root_dir}/LabelHand"
test_data_root = {
    "S2L1C": f"{default_root_dir}/S2L1CHand",
    "S1GRD": f"{default_root_dir}/S1GRDHand",
}
test_label_data_root = f"{default_root_dir}/LabelHand"

# Define split files because all samples are saved in the same folder.
train_split = f"{default_root_dir}/splits/flood_train_data.txt"
val_split = f"{default_root_dir}/splits/flood_valid_data.txt"
test_split = f"{default_root_dir}/splits/flood_test_data.txt"

# Define suffix, again using dicts.
img_grep = {
    "S2L1C": "*_S2Hand.tif",
    "S1GRD": "*_S1Hand.tif",
}
label_grep = "*_LabelHand.tif"

# With TerraTorch, you can select a subset of the dataset bands as model inputs by providing dataset_bands (all bands in the data) and output_bands (selected bands). This setting is optional for all modalities and needs to be provided as dicts.
# Here is an example for with S-1 GRD. You could change the output to ["VV"] to only train on the first band. Note that means and stds must be aligned with the output_bands (equal length of values). 
dataset_bands = {
    "S1GRD": ["VV", "VH"]
}
output_bands = {
    "S1GRD": ["VV", "VH"]
}

no_label_replace = -1,  # Replace NaN labels. defaults to -1 which is ignored in the loss and metrics.
no_data_replace = 0,  # Replace NaN data

terramind_backbone = 'terramind_v1_base' # choice of 'terramind_v1_base', 'terramind_v1_base_tim', 'terramind_v1_large', 'terramind_v1_large_tim'

max_epochs = 100

indices = [5, 11, 17, 23]
if 'terramind_v1_base' in terramind_backbone:
    indices = [2, 5, 8, 11]  # indices for terramind_v1_base
elif 'terramind_v1_large' in terramind_backbone: 
    indices = [5, 11, 17, 23]  # indices for terramind_v1_large

# Define standardization values. We use the pre-training values here and providing the additional modalities is not a problem, which makes it simple to experiment with different modality combinations. Alternatively, use the dataset statistics that you can generate using `terratorch compute_statistics -c config.yaml` (requires concat_bands: true for this multimodal datamodule).
data_mean = {
    "S2L1C": [2357.089, 2137.385, 2018.788, 2082.986, 2295.651, 2854.537, 3122.849, 3040.560, 3306.481, 1473.847, 506.070, 2472.825, 1838.929],
    "S2L2A": [1390.458, 1503.317, 1718.197, 1853.910, 2199.100, 2779.975, 2987.011, 3083.234, 3132.220, 3162.988, 2424.884, 1857.648],
    "S1GRD": [-12.599, -20.293],
    "S1RTC": [-10.93, -17.329],
    "RGB": [87.271, 80.931, 66.667],
    "DEM": [670.665]
}

data_stds = {
    "S2L1C": [1624.683, 1675.806, 1557.708, 1833.702, 1823.738, 1733.977, 1732.131, 1679.732, 1727.26, 1024.687, 442.165, 1331.411, 1160.419],
    "S2L2A": [2106.761, 2141.107, 2038.973, 2134.138, 2085.321, 1889.926, 1820.257, 1871.918, 1753.829, 1797.379, 1434.261, 1334.311],
    "S1GRD": [5.195, 5.890],
    "S1RTC": [4.391, 4.459],
    "RGB": [58.767, 47.663, 42.631],
    "DEM": [951.272],
}

model_factory="EncoderDecoderFactory",  # Combines a backbone with necks, the decoder, and a head

remove_cls_token = False # TerraMind is trained without CLS token, which neads to be specified.

freeze_backbone = True, # Only used to speed up fine-tuning in this demo, we highly recommend fine-tuning the backbone for the best performance. 
freeze_decoder = False,  # Should be false in most cases as the decoder is randomly initialized.
class_names = ["Others", "Water"]  # optionally define class names

In [ ]:
# Trainer specific configurations
config['trainer']['default_root_dir'] = default_root_dir
config['trainer']['max_epochs'] = max_epochs
config['trainer']['callbacks'][-1]['init_args']['dirpath'] = checkpoint_path

# Model specific configurations
config['model']['init_args']['model_args']['necks'][0]['indices'] = indices
config['model']['init_args']['model_args']['backbone'] = terramind_backbone
config['model']['init_args']['model_factory'] = model_factory
config['model']['init_args']['class_names'] = class_names
config['model']['init_args']['freeze_decoder'] = freeze_decoder
config['model']['init_args']['freeze_backbone'] = freeze_backbone

for index, neck in enumerate(config['model']['init_args']['necks']):
    if neck['name'] == 'ReshapeTokensToImage':
        config['model']['init_args']['necks'][index]['remove_cls_token'] = remove_cls_token
    elif neck['name'] == 'LearnedInterpolateToPyramidal':
        # Some decoders like UNet or UperNet expect hierarchical features. 
        # Therefore, we need to learn a upsampling for the intermediate embedding layers when using a ViT like TerraMind.
        continue


# Data specific configurations
config['data']['init_args']['train_data_root'] = train_data_root
config['data']['init_args']['test_data_root'] = test_data_root
config['data']['init_args']['val_data_root'] = val_data_root

config['data']['init_args']['train_split'] = train_split
config['data']['init_args']['val_split'] = val_split
config['data']['init_args']['test_split'] = test_split

config['data']['init_args']['modalities'] = modalities

config['data']['init_args']['label_grep'] = label_grep
config['data']['init_args']['image_grep'] = img_grep

config['data']['init_args']['num_classes'] = num_classes
config['data']['init_args']['no_data_replace'] = no_data_replace
config['data']['init_args']['no_label_replace'] = no_label_replace

config['data']['init_args']['means'] = data_mean 
config['data']['init_args']['stds'] = data_stds

config['data']['init_args']['batch_size'] = batch_size
config['data']['init_args']['num_workers'] = num_workers

config['data']['init_args']['num_classes'] = num_classes

In [ ]:
# Rename configuration file name to user specific filename
import os

config_filename = f"{identifier}-flood_terramind.yaml"
config_filepath = f"../configs/{config_filename}"
with open(config_filepath, 'w') as config_file:
    yaml.dump(config, config_file, default_flow_style=False)

# Upload config files to s3 bucket
configs = sagemaker_session.upload_data(path=config_filepath, bucket=BUCKET_NAME, key_prefix='data/configs')

## Sen1Floods11 Dataset

Lets start with analysing the dataset

TerraTorch provides generic data modules that work directly with PyTorch Lightning.

Sen1Floods11 is a multimodal dataset that provides Sentinel-2 L2A and Sentinel-1 GRD data. 
Therefore, we are using the `GenericMultiModalDataModule`. 
This module is similar to the `GenericNonGeoSegmentationDataModule`, which is used for standard segmentation tasks.
However, the data roots, `img_grep` are other settings are provided as dict to account for the multimodal inputs. You find all settings in the [documentation](https://ibm.github.io/terratorch/stable/generic_datamodules/). 
In a Lightning config, the data module is defined with the `data` key.

In [ ]:
# Helper function for plotting both modalities
def plot_sample(sample):
    s1 = sample['image']['S1GRD'].cpu().numpy()
    s2 = sample['image']['S2L1C'].cpu().numpy()
    mask = sample['mask'].cpu().numpy()
    
    # Scaling data. Using -30 to 0 scaling for S-1 and 0 - 2000 for S-2. S-1 is visualized as [VH, VV, VH]
    s1 = (s1.clip(-30, 0) / 30 + 1) * 255
    s2 = (s2.clip(0, 2000) / 2000) * 255
    s1_rgb = np.stack([s1[1], s1[0], s1[1]], axis=0).astype(np.uint8).transpose(1,2,0)
    s2_rgb = s2[[3,2,1]].astype(np.uint8).transpose(1,2,0)
    
    fig, ax = plt.subplots(1, 3, figsize=(12, 4))
    ax[0].imshow(s1_rgb)
    ax[0].set_title('S-1 GRD')
    ax[0].axis('off')
    ax[1].imshow(s2_rgb)
    ax[1].set_title('S-2 GRD')
    ax[1].axis('off')   
    ax[2].imshow(mask, vmin=-1, vmax=1, interpolation='nearest')
    ax[2].set_title('Mask')
    ax[2].axis('off')
    fig.tight_layout()
    plt.show()

In [ ]:
# plot some samples
plot_sample()
plot_sample()
plot_sample()

# Fine-tune TerraMind via PyTorch Lightning

With TerraTorch, we can use standard Lightning components for the fine-tuning.
These include callbacks and the trainer class.
TerraTorch provides EO-specific tasks that define the training and validation steps.
In this case, we are using the `SemanticSegmentationTask`.
We refer to the [TerraTorch paper](https://arxiv.org/abs/2503.20563) for a detailed explanation of the TerraTorch tasks.

In [ ]:
# Setup variables for training using sagemaker

name = f'{identifier}-sagemaker'
role = get_execution_role()
input_s3_uri = f"s3://{BUCKET_NAME}/data"
model_name = f"{identifier}-workshop.ckpt",

environment_variables = {
    'CONFIG_FILE': f"{base_path}/configs/{config_filename}",
    'MODEL_DIR': model_path,
    'MODEL_NAME': model_name,
    'S3_URL': input_s3_uri,
    'ROLE_ARN': role,
    'ROLE_NAME': role.split('/')[-1],
    'EVENT_TYPE': usecase,
    'VERSION': 'v1'
}

ecr_container_url = '637423382292.dkr.ecr.us-west-2.amazonaws.com/eo_training:latest'
sagemaker_role = 'SageMaker-ExecutionRole-20240206T151814'

instance_type = 'ml.p3.2xlarge'

instance_count = 1
memory_volume = 50


In [ ]:

# Establish an estimator (model) using sagemaker and the configurations from the previous cell.
estimator = Estimator(image_uri=ecr_container_url,
                      role=get_execution_role(),
                      base_job_name=name,
                      instance_count=1,
                      environment=environment_variables,
                      instance_type=instance_type)

estimator.fit()

In [ ]:

# Save important values in a file for reuse.
export_values = {
    'identifier': identifier,
    'model_name': model_name,
    'config_filename': config_filename,
    'bucket_name': BUCKET_NAME
}

with open('../variables.yaml', 'w') as variable_export:
    yaml.dump(export_values, variable_export, default_flow_style=False)


# Test the fine-tuned model 

In [ ]:
# Let's test the fine-tuned model
best_ckpt_path = f"{output_folder}/terramind_base_sen1floods11/checkpoints/best-mIoU.ckpt"
trainer.test(model, datamodule=datamodule, ckpt_path=best_ckpt_path)

# Note: This demo only trains for 5 epochs by default, which does not result in good test metrics.

In [ ]:
# Now we can use the model for predictions and plotting
model = terratorch.tasks.SemanticSegmentationTask.load_from_checkpoint(
    best_ckpt_path,
    model_factory=model.hparams.model_factory,
    model_args=model.hparams.model_args,
)

test_loader = datamodule.test_dataloader()
with torch.no_grad():
    batch = next(iter(test_loader))
    images = batch["image"]
    for mod, value in images.items():
        images[mod] = value.to(model.device)
    masks = batch["mask"].numpy()

    with torch.no_grad():
        outputs = model(images)
    
    preds = torch.argmax(outputs.output, dim=1).cpu().numpy()

for i in range(4):
    sample = {
        "image": batch["image"]["S2L1C"][i].cpu(),
        "mask": batch["mask"][i],
        "prediction": preds[i],
    }
    test_dataset.plot(sample)
    plt.show()
    
# Note: This demo only trains for 5 epochs by default, which does not result in good predictions.